In [231]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import shutil
import random

from scipy import signal
from collections import OrderedDict

In [232]:
path = r'c:\Users\dhers\OneDrive - UBA\video_examples_dlc3\2023-12_Tg-3m_TS_R16_C12-n_A_LDLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900.h5'

df = pd.read_hdf(path)
scorer = df.columns.levels[0][0]
bodyparts = df.columns.levels[1].to_list()

df_raw = df.copy()
df

scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900              \
bodyparts                                          obj_1               
coords                                                 x           y   
0                                             228.342728  122.385658   
1                                             228.252258  122.206512   
2                                             228.395233  122.252884   
3                                             228.356918  122.219719   
4                                             228.221466  122.089645   
...                                                  ...         ...   
7495                                          227.287048  123.666252   
7496                                          227.550049  123.478401   
7497                                          227.542114  123.653694   
7498                                          227.606750  123.541985   
7499                                          227.499817  123.397171   

scorer                                                               \
bodyparts                  obj_2                               nose   
coords    likelihood           x           y likelihood           x   
0           0.678950  490.379974  122.702538   0.967827  479.936859   
1           0.680715  490.438782  122.581024   0.967655  479.865051   
2           0.680249  490.423340  122.614189   0.971328  479.895599   
3           0.689105  490.427612  122.598946   0.970381  479.884247   
4           0.668227  490.466736  122.526947   0.968033  479.802887   
...              ...         ...         ...        ...         ...   
7495        0.883922  228.830399  123.489098   0.117463  508.896973   
7496        0.881149  229.131760  123.334778   0.128264  499.909790   
7497        0.884025  229.105957  123.493416   0.110442  487.750427   
7498        0.874062  229.211823  123.411804   0.131441  478.001984   
7499        0.854557  229.107147  123.263474   0.167002  472.220428   

scorer                                        ...                         \
bodyparts                              L_ear  ...       body      tail_1   
coords              y likelihood           x  ... likelihood           x   
0          117.071899   0.018626  223.810791  ...   0.011640  245.853455   
1          116.988678   0.019303  225.331848  ...   0.011735  245.910645   
2          117.029205   0.018515  225.361237  ...   0.011523  245.940872   
3          117.013489   0.018538  225.357437  ...   0.011493  245.943481   
4          116.945145   0.018778  225.337189  ...   0.012223  245.960999   
...               ...        ...         ...  ...        ...         ...   
7495       223.981522   0.857031  519.670654  ...   0.966113  457.936310   
7496       223.745819   0.908773  513.468140  ...   0.986860  460.171082   
7497       219.406433   0.919292  507.801910  ...   0.987153  462.286957   
7498       217.303238   0.850866  503.347778  ...   0.969723  464.240204   
7499       211.365860   0.943808  500.405029  ...   0.972605  465.092987   

scorer                                                               \
bodyparts                             tail_2                          
coords              y likelihood           x           y likelihood   
0          117.518501   0.009870  485.927734  125.908264   0.010542   
1          117.514397   0.009834  485.940247  125.905396   0.010693   
2          117.514595   0.009914  485.939270  125.889511   0.010505   
3          117.525215   0.009772  485.924133  125.886253   0.010398   
4          117.451729   0.010238  222.245895  118.126045   0.010143   
...               ...        ...         ...         ...        ...   
7495        79.976410   0.901801  415.288605   46.331703   0.931013   
7496        79.254478   0.908870  416.911163   45.702579   0.939019   
7497        78.804512   0.949498  418.767029   43.599628   1.000000   
7498        78.469414   0.965404  421.761902   42.516113   0.989529   
7499    

In [233]:
# Try different filtering parameters
med_window = 3
sigma, n_sigmas = 0.6, 2
N = int(2 * n_sigmas * sigma + 1)

# Gaussian kernel
gauss_kernel = signal.windows.gaussian(N, sigma)
gauss_kernel = gauss_kernel / sum(gauss_kernel)

pad_width = (len(gauss_kernel) - 1) // 2
gauss_kernel

array([0.16637851, 0.66724298, 0.16637851])

In [234]:
# Iterate over each body part
for point in bodyparts:

    mean = df[scorer, point, 'likelihood'].mean()
    std_dev = df[scorer, point, 'likelihood'].std()
    limit = mean - 2*std_dev

    print(f'{point}\t{limit}')
    
    # Create mask where likelihood is below the threshold
    mask = df[scorer, point, 'likelihood'] < limit
    # Set 'x' and 'y' to NaN where likelihood is below the threshold
    df.loc[mask, (scorer, point, ['x', 'y'])] = np.nan

    # Iterate over x and y axes
    for axis in ['x', 'y']:
        # Access the specific axis column for this body part
        # column = df[scorer, point, axis]

        # Check if there are any non-NaN values left to interpolate
        if df[scorer, point, axis].notna().sum() > 1:
            # Interpolate using the pchip method, limit_area='inside' ensures interpolation only within valid data
            df[scorer, point, axis] = df[scorer, point, axis].interpolate(method='pchip', limit_area='inside')
            
            # Forward fill the remaining NaN values
            df[scorer, point, axis] = df[scorer, point, axis].ffill()
            
            # Apply median filter
            df[scorer, point, axis] = signal.medfilt(df[scorer, point, axis], kernel_size=med_window)
            
            # Pad the median filtered data to mitigate edge effects
            padded = np.pad(df[scorer, point, axis], pad_width, mode='edge')
            # Apply convolution
            smooth = signal.convolve(padded, gauss_kernel, mode='valid')
            # Trim the padded edges to restore original length
            df[scorer, point, axis] = smooth[:len(df[scorer, point, axis])]
            
            if 'obj' in point:
                df[scorer, point, axis] = df[scorer, point, axis].median()
            


df

L_ear	0.6587399691343307
R_ear	0.6661268770694733
body	0.690492108464241
head	0.6923242062330246
neck	0.6815532296895981
nose	0.6498726904392242
obj_1	0.0021503567695617676
obj_2	0.5591788589954376
tail_1	0.6267890632152557
tail_2	0.4921550154685974
tail_3	0.3618008494377136


scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900              \
bodyparts                                          obj_1               
coords                                                 x           y   
0                                              228.25975  122.322767   
1                                              228.25975  122.322767   
2                                              228.25975  122.322767   
3                                              228.25975  122.322767   
4                                              228.25975  122.322767   
...                                                  ...         ...   
7495                                           228.25975  122.322767   
7496                                           228.25975  122.322767   
7497                                           228.25975  122.322767   
7498                                           228.25975  122.322767   
7499                                           228.25975  122.322767   

scorer                                                               \
bodyparts                  obj_2                               nose   
coords    likelihood           x           y likelihood           x   
0           0.678950  489.839877  123.395511   0.967827         NaN   
1           0.680715  489.839877  123.395511   0.967655         NaN   
2           0.680249  489.839877  123.395511   0.971328         NaN   
3           0.689105  489.839877  123.395511   0.970381         NaN   
4           0.668227  489.839877  123.395511   0.968033         NaN   
...              ...         ...         ...        ...         ...   
7495        0.883922  489.839877  123.395511   0.117463  509.613698   
7496        0.881149  489.839877  123.395511   0.128264  499.382007   
7497        0.884025  489.839877  123.395511   0.110442  488.151552   
7498        0.874062  489.839877  123.395511   0.131441  478.661989   
7499        0.854557  489.839877  123.395511   0.167002  473.182355   

scorer                                        ...                         \
bodyparts                              L_ear  ...       body      tail_1   
coords              y likelihood           x  ... likelihood           x   
0                 NaN   0.018626         NaN  ...   0.011640         NaN   
1                 NaN   0.019303         NaN  ...   0.011735         NaN   
2                 NaN   0.018515         NaN  ...   0.011523         NaN   
3                 NaN   0.018538         NaN  ...   0.011493         NaN   
4                 NaN   0.018778         NaN  ...   0.012223         NaN   
...               ...        ...         ...  ...        ...         ...   
7495       224.164478   0.857031  519.503118  ...   0.966113  458.604931   
7496       223.063054   0.908773  513.557366  ...   0.986860  460.218246   
7497       219.778487   0.919292  508.003577  ...   0.987153  462.259899   
7498       216.665312   0.850866  503.599240  ...   0.969723  463.915226   
7499       212.353712   0.943808  500.894639  ...   0.972605  464.240204   

scorer                                                                         \
bodyparts                            tail_2                            tail_3   
coords             y likelihood           x          y likelihood           x   
0                NaN   0.009870         NaN        NaN   0.010542         NaN   
1                NaN   0.009834         NaN        NaN   0.010693         NaN   
2                NaN   0.009914         NaN        NaN   0.010505         NaN   
3                NaN   0.009772         NaN        NaN   0.010398         NaN   
4                NaN   0.010238         NaN        NaN   0.010143         NaN   
...              ...        ...         ...        ...        ...         ...   
7495       79.216542   0.901801  415.558564  45.563619   0.931013  366.212239   
7496       79.179614   0.908870  416.949981  45.352694   0.939019  367.317787   
7497       78.823624   0.949498  418.956

In [235]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df[scorer]['nose']['x']))
fig.add_trace(go.Scatter(x=df_raw.index, y=df_raw[scorer]['nose']['x']))

In [236]:
# Save the updated DataFrame to an HDF5 file
out_path = r'c:\Users\dhers\OneDrive - UBA\video_examples_dlc3\2023-12_Tg-3m_TS_R16_C12-n_A_LDLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900.h5'
# df.to_hdf(out_path, key='df', mode='w')